## Scrape URLs

In [14]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from datetime import datetime
import pandas as pd
import json
import pymongo
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from IPython.display import clear_output

In [15]:
maxLen = 200

In [16]:
def create_ListofDict(u,h,d,t):
    listofdict = []
    for c,lx in enumerate(u,0):
        info = {
            "timestamp": t,
            "URL":    lx,
            "headline": h[c],
            "newsdate":  d[c]
        }
        listofdict.append(info)
    return listofdict

In [54]:
def create_ListofDictdf(df):
    listofdict = []
    for c,lx in enumerate(df.URL,0):
        info = {
            "timestamp": df.timestamp[c],
            "URL":    lx,
            "headline": df.headline[c],
            "newsdate":  df.newsdate[c]
        }
        listofdict.append(info)
    return listofdict

In [17]:
def convertTHMonth(monthString):
    switcher = {
        "มกราคม": 1,
        "กุมภาพันธ์": 2,
        "มีนาคม": 3,
        "เมษายน":  4,
        "พฤษภาคม": 5,
        "มิถุนายน": 6,
        "กรกฎาคม": 7,
        "สิงหาคม": 8,
        "กันยายน": 9,
        "ตุลาคม": 10,
        "พฤศจิกายน": 11,
        "ธันวาคม": 12}
    return switcher.get(monthString)

In [55]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_c_urlList = mydb["URL_LIST_CRIME"]
q_p_urlList = mydb["URL_LIST_POLITIC"]
i_p2_urlList = mydb["URL_LIST_POLITIC_REMDUP"]
q_e_urlList = mydb["URL_LIST_ENTER"]

### Scrape URL

In [19]:
urls_crime = 'https://www.dailynews.co.th/crime?page='
urls_political = 'https://www.dailynews.co.th/politics?page='
urls_entertain = 'https://www.dailynews.co.th/entertainment?page='

In [ ]:
q_c_urlList.delete_many({})

for e in range(1,maxLen):
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_crime+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    urlList,dtList,hList = [],[],[]
    
    for div in soup.find_all("div", class_=lambda value: value and value=="left"):
        for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
            urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
            for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                dstr = s.text.strip().split()
                dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
            for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                hList.append(str(h.text.strip()))
    process_data = create_ListofDict(urlList,hList,dtList,today)
    q_c_urlList.insert_many(process_data)
    time.sleep(2)
    driver.quit()

In [ ]:
#q_p_urlList.delete_many({})

for e in range(574,583):
    print(e)
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_political+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    try:
        urlList,dtList,hList = [],[],[]
        myElem = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CLASS_NAME, 'left')))
        time.sleep(4)
        for div in soup.find_all("div", class_=lambda value: value and value=="left"):
            for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
                urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
                for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                    dstr = s.text.strip().split()
                    dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                    dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
                for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                    hList.append(str(h.text.strip()))
        
        process_data = create_ListofDict(urlList,hList,dtList,today)
        q_p_urlList.insert_many(process_data)
    except TimeoutException:
        print('Request time out')
    driver.quit()
    clear_output(wait=True)

580


In [51]:
cursor2 = q_p_urlList.find()
df_politic_url = pd.DataFrame(cursor2)
df_politic_url = df_politic_url[(df_politic_url.newsdate >= '2020-07-01 00:00:00')]
df_politic_url.drop_duplicates(subset=['URL'], inplace=True, ignore_index=True)

In [52]:
df_politic_url.shape

(5526, 5)

In [53]:
df_politic_url.tail()

,_id,timestamp,URL,headline,newsdate
5521,5fa92265019e3d90e89381a0,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788743,รมว.ศธ.ชงทดลองเปิดเรียน100%ดีเดย์13ส.ค.,2020-08-07 15:35:00
5522,5fa92265019e3d90e89381a1,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788742,สบส.สั่งปิด2คลินิกโกงงบบัตรทอง ขู่ถอนใบอนุญาต,2020-08-07 15:30:00
5523,5fa92265019e3d90e89381a2,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788741,สธ.เผยผลตรวจทหารสหรัฐ110รายไม่พบโควิด-19,2020-08-07 15:28:00
5524,5fa92265019e3d90e89381a3,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788738,เตือน!คนรุ่นใหม่ลุกฮือสู้เผด็จการ ล่า5หมื่นชื่...,2020-08-07 15:21:00
5525,5fa92265019e3d90e89381a4,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788726,"""ชวน""เตือนสังคมไทยอย่าเกรงใจจนทำสิ่งไม่ถูกต้อง",2020-08-07 15:16:00


In [56]:
process_data = create_ListofDictdf(df_politic_url)
i_p2_urlList.insert_many(process_data)

In [ ]:
q_e_urlList.delete_many({})

for e in range(1,maxLen):
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_entertain+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    urlList,dtList,hList = [],[],[]
    
    for div in soup.find_all("div", class_=lambda value: value and value=="left"):
        for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
            urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
            for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                dstr = s.text.strip().split()
                dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
            for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                hList.append(str(h.text.strip()))
    process_data = create_ListofDict(urlList,hList,dtList,today)
    q_e_urlList.insert_many(process_data)
    time.sleep(2)
    driver.quit()